In [ ]:

import pandas as pd
import torch
import random
from transformers import GPT2TokenizerFast
from transformers import GPT2LMHeadModel

backbone = "its5Q/rugpt3large_mailqa"
device = "cuda"
model = GPT2LMHeadModel.from_pretrained(backbone).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(backbone, use_fast=True)


In [2]:
def ans(question, description='', category=''):
    seed = random.randint(1, 10000000)
    print(f'Seed: {seed}')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    inp = tokenizer.encode(f'{"Категория: " + category + "\n" if category else ""}Вопрос: {question}\nОписание: {description}\nОтвет:',return_tensors="pt").to(device)
    print('question',question)
    gen = model.generate(inp, do_sample=True, top_p=0.9, temperature=0.86, max_new_tokens=100, repetition_penalty=1.2) #, stop_token="<eos>")
    
    gen = tokenizer.decode(gen[0])
    return gen[:gen.index('<eos>') if '<eos>' in gen else len(gen)]

In [19]:
print(ans("Как какать?", ""))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Seed: 8387994
question Как какать?
Вопрос: Как какать?
Описание: 
Ответ: не надо.... если уже не хочешь....
значит, пора начинать новую жизнь!!!
а по вопросу-неправильно написал!)))) ) у меня вообще в последнее время редко какает... но я думаю через 3 дня должен наступить этот момент, если до сих пор нет стула))))))) 

Вопрос: Как какать?
Описание: 
Ответ: не надо.... если уже не хочешь....
значит, пора начинать новую жизнь!!!
а по вопросу-неправильно написал!)))) ) у меня вообще в последнее время редко какает... но я думаю через 3 дня должен наступить этот момент, если до сих пор нет стула))))))) 



In [ ]:
%pip install pyTelegramBotAPI

In [ ]:
from time import sleep
import telebot
import random

BOT_TOKEN = ''
bot = telebot.TeleBot(BOT_TOKEN)

@bot.message_handler(commands=['help', 'start'])
def say_welcome(message):
    bot.send_message(message.chat.id,
                     'Добро пожаловать QA bot, задайте вопрос через /question, а я отвечу...',
                     parse_mode='html')
        
@bot.message_handler(commands=['question'])
def generate(message):
    text = ' '.join(message.text.split()[1:])
    print(text)

    reply = ans(text, "")
   
    bot.send_message(message.chat.id, reply)
    #logger.info(f'{reply}')


if __name__ == '__main__':
    bot.polling(none_stop=True)
